In [2]:

import requests
import json
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

True

# Transaction Classification based on Merchant

In [6]:
url = "https://api.ai71.ai/v1/chat/completions"
AI71_TOKEN = os.getenv('AI71_TOKEN')
role_content = "You will be provided with a company name, and your task is to classify it to fitness, food, healthcare, clothing, transportation, fun, utilities, personal care, shopping, miscellaneous"
user_content = "CLASSPASS* MONTHLY CLASSPASSCOM"

payload = json.dumps({
  "model": "tiiuae/falcon-180b-chat",
  "messages": [
    {
      "role": "system",
      "content": role_content
    },
    {
      "role": "user",
      "content": user_content
    }
  ]
})
headers = {
  'Content-Type': 'application/json',
  'Authorization': f'Bearer {AI71_TOKEN}'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"id":"RequestType.CHAT_COMPLETION-tiiuae/falcon-180b-chat-2ffbf6db-b047-4dab-b09c-bfd60696f919-api71-api-2ffbf6db-b047-4dab-b09c-bfd60696f919-de96dedb-a51b-4dfd-be10-eb0ab9bf8f0d","created":1722060829,"model":"tiiuae/falcon-180b-chat","usage":{"completion_tokens":2,"prompt_tokens":60,"total_tokens":62},"choices":[{"finish_reason":"stop","index":0,"message":{"content":" Fitness","role":"assistant"}}],"object":"chat.completion"}


In [7]:
try:
    response_json = response.json()
    choices = response_json.get('choices', [])
    if choices:
        message_content = choices[0].get('message', {}).get('content', '')
        print(f'Classification Result: {message_content.strip()}')
    else:
        print("No choices found in the response.")
        
except ValueError:
    print("Response is not in JSON format:")
    print(response.text)

Classification Result: Fitness


In [10]:
df = pd.read_csv('../data/sample_transactions.csv')
df.head()

,Date,Description,Merchant,Amount
0,07 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,8.70
1,07 Jul 2024,CARD NO.4439********1246 AL AQSA GAR AND BLANK...,AL AQSA GAR AND BLANKE,85.00
2,07 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,31.55
3,05 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,31.39
4,05 Jul 2024,CARD NO.4439********1246 Talabat DUBAI:AE 0835...,Talabat,139.63


In [43]:
import os
import json
import requests

def classify_company(user_content):
    url = "https://api.ai71.ai/v1/chat/completions"
    AI71_TOKEN = os.getenv('AI71_TOKEN')
    role_content = (
        "You will be provided with a company name, and your task is to classify it to one of the following "
        "fitness, groceries, restaurants and cafes, healthcare, clothing, jewelry, transportation, phone and internet, miscellaneous, others"
    )
    
    payload = json.dumps({
        "model": "tiiuae/falcon-180b-chat",
        "messages": [
            {
                "role": "system",
                "content": role_content
            },
            {
                "role": "user",
                "content": user_content
            }
        ]
    })
    
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {AI71_TOKEN}'
    }
    
    response = requests.post(url, headers=headers, data=payload)

    try:
        response_json = response.json()
        choices = response_json.get('choices', [])
        if choices:
            result = choices[0].get('message', {}).get('content', '').strip()
            return result
        else:
            return None
            
    except ValueError:
        return None

 
classify_company("CLASSPASS* MONTHLY CLASSPASSCOM")

'fitness'

In [44]:
df['Categoory'] = df['Merchant'].apply(classify_company)

In [45]:
df

,Date,Description,Merchant,Amount,Categoory
0,07 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,8.70,groceries
1,07 Jul 2024,CARD NO.4439********1246 AL AQSA GAR AND BLANK...,AL AQSA GAR AND BLANKE,85.00,Clothing
2,07 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,31.55,groceries
3,05 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,31.39,groceries
4,05 Jul 2024,CARD NO.4439********1246 Talabat DUBAI:AE 0835...,Talabat,139.63,"Talabat is a food delivery service, so it shou..."
5,05 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,50.98,groceries
6,04 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,17.25,groceries
7,03 Jul 2024,CARD NO.4439********1246 Talabat DUBAI:AE 6897...,Talabat,66.04,"Talabat is a food delivery service, so it shou..."
8,03 Jul 2024,CARD NO.4439********1246 Amazon.ae Dubai:AE 50...,Amazon.ae,175.25,Amazon.ae is an e-commerce platform that sells...
9,02 Jul 2024,CARD NO.4439********1246 UNION COOP DUBAI:AE 2...,UNION COOP,5.50,groceries
